# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

rice = cv2.imread('rice.png', cv2.IMREAD_GRAYSCALE)
bin_okna = np.zeros_like(rice)
rice_x, rice_y = rice.shape

w_size = 16
local_thrs = np.zeros((rice_x // w_size, rice_y // w_size))

for j in range(0, rice_x // w_size):
    j_2 = j * w_size
    for i in range(0, rice_y // w_size):
        i_2 = i * w_size
        local_image = rice[i_2:i_2 + w_size, j_2:j_2 + w_size]
        local_thrs[i, j] = np.mean(local_image)

for j in range(rice_x):
    for i in range(rice_y):
        bin_okna[i, j] = rice[i, j] > local_thrs[i // w_size, j // w_size]


fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(bin_okna, cmap='gray')
ax.axis('off')

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
rice = cv2.imread('rice.png', cv2.IMREAD_GRAYSCALE)
bin_okna_inter = np.zeros_like(rice)
rice_y, rice_x = rice.shape

w_size = 16
half_w = w_size // 2

local_thrs = np.zeros((rice_x // w_size, rice_y // w_size))
thr_y, thr_x = local_thrs.shape

for j in range(0, rice_x // w_size):
    j_2 = j * w_size
    for i in range(0, rice_y // w_size):
        i_2 = i * w_size
        local_image = rice[i_2:i_2 + w_size, j_2:j_2 + w_size]
        local_thrs[i, j] = np.mean(local_image)

for i in range(rice_y):
    iT = i // w_size
    iT_u = iT - 1 if iT > 0 else 0
    iT_d = iT + 1 if iT < thr_y - 1 else thr_y - 1
    for j in range(rice_x):
        jT = j // w_size
        jT_l = jT - 1 if jT > 0 else 0
        jT_r = jT + 1 if jT < thr_x - 1 else thr_x - 1

        t11 = np.mean([local_thrs[iT, jT], local_thrs[iT_u, jT_l], local_thrs[iT, jT_l], local_thrs[iT_u, jT]])
        t12 = np.mean([local_thrs[iT, jT], local_thrs[iT_u, jT_r], local_thrs[iT, jT_r], local_thrs[iT_u, jT]])
        t21 = np.mean([local_thrs[iT, jT], local_thrs[iT_d, jT_l], local_thrs[iT, jT_l], local_thrs[iT_d, jT]])
        t22 = np.mean([local_thrs[iT, jT], local_thrs[iT_d, jT_r], local_thrs[iT, jT_r], local_thrs[iT_d, jT]])

        dX1 = j - (jT * w_size + half_w)
        dX2 = (jT + 1) * w_size + half_w - j
        dY1 = i - (iT * w_size + half_w)
        dY2 = (iT + 1) * w_size + half_w - i

        interpol_x1 = (t11 * dX2 + t12 * dX1) / (dX2 + dX1)
        interpol_x2 = (t21 * dX2 + t22 * dX1) / (dX2 + dX1)
        interpolated_thr = (interpol_x1 * dY2 + interpol_x2 * dY1) / (dY2 + dY1)

        bin_okna_inter[i, j] = rice[i, j] > interpolated_thr

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(bin_okna_inter, cmap='gray')
ax.axis('off')

In [ ]:
# KODY Z ZADANIA ROBIONEGO NA LABORATORIACH DO PORÓWNANIA METOD

def otsu(I):

    hist = cv2.calcHist([I], [0], None, [256], [0, 256])
    hist = np.squeeze(hist)
    hist = hist / sum(hist)
    intensity_levels = np.arange(256)
    m_g = np.dot(intensity_levels, hist)
    sigma_2_B = np.zeros(256)

    P_0 = 0
    m_k = 0

    for k in range(256):
        P_0 += hist[k]
        m_k += k * hist[k]

        if 0 < P_0 < 1:
            sigma_2_B[k] = ((m_g*P_0 - m_k)**2)/(P_0*(1-P_0))
        else:
            sigma_2_B[k] = 0

    thr = np.argmax(sigma_2_B)

    binary_image = (I > thr).astype(np.uint8) * 255

    return binary_image

def local_bin(image, window_size):
    (X, Y) = image.shape
    binary_image = np.zeros((X, Y), dtype=np.uint8)

    half_window = window_size // 2

    for j in range(half_window, Y-half_window):
        for i in range(half_window, X-half_window):
            local_image = image[i - half_window:i + half_window + 1, j - half_window: j + half_window + 1]
            local_thr = np.mean(local_image)
            binary_image[i, j] = 255 if image[i, j] > local_thr else 0

    return binary_image

def local_bin_SP(image, window_size, k=0.15, R=128):
    (X, Y) = image.shape
    binary_image = np.zeros((X, Y), dtype=np.uint8)

    half_window = window_size // 2

    for i in range(half_window, X - half_window):
        for j in range(half_window, Y - half_window):
            local_image = image[i - half_window:i + half_window + 1, j - half_window: j + half_window + 1]
            local_mean = np.mean(local_image)
            local_std = np.std(local_image)
            local_thr = local_mean * (1 - k * ((local_std / R) - 1)) if local_std > 0 else local_mean
            binary_image[i, j] = 255 if image[i, j] > local_thr else 0

    return binary_image

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15, 10))

ax[0, 0].imshow(rice, cmap='gray')
ax[0, 0].axis('off')
ax[0, 0].set_title('Obraz orginalny')

ax[0, 1].imshow(bin_okna, cmap='gray')
ax[0, 1].axis('off')
ax[0, 1].set_title('lokalna w oknach')

ax[0, 2].imshow(bin_okna_inter, cmap='gray')
ax[0, 2].axis('off')
ax[0, 2].set_title('lokalna w oknach z interpolacją')

bin_otsu = otsu(rice)
ax[1, 0].imshow(bin_otsu, cmap='gray')
ax[1, 0].axis('off')
ax[1, 0].set_title('Otsu')

bin_local = local_bin(rice, 16)
ax[1, 1].imshow(bin_local, cmap='gray')
ax[1, 1].axis('off')
ax[1, 1].set_title('Local')

bin_local_SP = local_bin_SP(rice, 16)
ax[1, 2].imshow(bin_local_SP, cmap='gray')
ax[1, 2].axis('off')
ax[1, 2].set_title('Local Sauvoli')

Różne metody binaryzacji dają różne efekty.

Metoda binaryzacji lokalnej w oknach działa w miarę dobrze ale posiada ona bardzo dużo artefaktów na obrazie wynikowym. Używając interpolacji między krawędziami ramek jesteśmy w stanie się pozbyć większości tych zakłóceń ale nie wszystkich.

Metoda Otsu dobrze dobiera próg, ale jest on globalny więc nie nadaje się do zdjęć z różnym stopniem oświetlenia lub połyskiem.

Metoda lokalna działa poprawnie na krawędziach obiektów, ale jednocześnie nie radzi sobie zbyt dobrze z tłem. Jej działanie poprawia metoda Sauvoli, która biorąc pod uwagę odchylenie standardowe lepiej wypełnia tło.